---

### **Import SK packages**

In [1]:
#r "nuget:Microsoft.SemanticKernel, 0.16.230615.1-preview"
#r "nuget:Microsoft.SemanticKernel.Connectors.Memory.Qdrant, 0.16.230615.1-preview"

Installed Packages Microsoft.SemanticKernel, 0.16.230615.1-preview Microsoft.SemanticKernel.Connectors.Memory.Qdrant, 0.16.230615.1-preview

---

### **Load secrets into local variables**

In [2]:
#!import ../Settings/AppConfig.cs

//#!import /Settings/AppConfig.cs

AppConfig.aoaiSettings apiConfig = AppConfig.GetSecrets();

// Get AOAI values
string mydeployDavinci = apiConfig.deployDavinci;
string mydeployEmbed = apiConfig.deployEmbed; 
string myendpoint = apiConfig.endpoint;
string myAPIKey = apiConfig.APIKey;
string myQdrantEndpoint = apiConfig.QdrantEndpoint;

//NOTE: If you run the below writeline command, be sure to clear the output before pushing to repo!!
//Console.WriteLine(mydeployDavinci + " & " + mydeployEmbed + " & " + myQdrantEndpoint + " & " + myendpoint + " & " + myAPIKey);

---
### **Build the Semantic Kernel client**

In [4]:
using Microsoft.SemanticKernel;
//using Microsoft.SemanticKernel.KernelExtensions;
//using System.IO;
//using Microsoft.SemanticKernel.Configuration;
//using Microsoft.SemanticKernel.SemanticFunctions;
//using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.Memory.Qdrant;

int qdrantVectorSize = 1536; // ada embeddings output dimensions
QdrantMemoryStore memoryStore = new(myQdrantEndpoint!, qdrantVectorSize);

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder
            // .WithLogger(ConsoleLogger.Log)
            .WithAzureTextCompletionService(mydeployDavinci!, myendpoint!, myAPIKey!)
            .WithAzureTextEmbeddingGenerationService(mydeployEmbed!, myendpoint!, myAPIKey!)
            .WithMemoryStorage(memoryStore)
            //.WithMemoryStorage(new VolatileMemoryStore())
            //.WithQdrantMemoryStore(Env.Var("QDRANT_ENDPOINT"), 1536) // This method offers an alternative approach to registering Qdrant memory store.
            .Build();

---
### **Create Collection and validate success**

In [6]:
const string memoryCollectionName = "Facts About Me";

var myBio = await kernel.Memory.SaveInformationAsync(memoryCollectionName, id: "LinkedIn Bio",
    text: "I currently work in the hotel industry at the front desk. I won the best team player award.");

var myHistory = await kernel.Memory.SaveInformationAsync(memoryCollectionName, id: "LinkedIn History",
    text: "I have worked as a tourist operator for 8 years. I have also worked as a banking associate for 3 years.");

var myRecentFB = await kernel.Memory.SaveInformationAsync(memoryCollectionName, id: "Recent Facebook Post",
    text: "My new dog Trixie is the cutest thing you've ever seen. She's just 2 years old.");

var myOldFB = await kernel.Memory.SaveInformationAsync(memoryCollectionName, id: "Old Facebook Post",
    text: "Can you believe the size of the trees in Yellowstone? They're huge! I'm so committed to forestry concerns.");

Console.WriteLine("Four GIGANTIC vectors were generated just now from those 4 pieces of text above.");

// check if collection was created

bool CollectionDoesExist = false;
CollectionDoesExist = await memoryStore.DoesCollectionExistAsync("Facts About Me").ConfigureAwait((false));
Console.WriteLine("Does the collection we just created exist? " + (CollectionDoesExist ? "Yes" :"No"));



Four GIGANTIC vectors were generated just now from those 4 pieces of text above.
Does the collection we just created exist? Yes


---
### **Retrieve Memories from Collection by keys**

In [7]:
Console.WriteLine("== Retrieving Memories Through the Kernel ==");
//MemoryQueryResult? lookup = await kernel.Memory.GetAsync(memoryCollectionName, "LinkedIn Bio");
var vdbResult1 = await kernel.Memory.GetAsync(memoryCollectionName, "LinkedIn Bio");
Console.WriteLine(vdbResult1 != null ? vdbResult1.Metadata.Text : "ERROR: memory not found");

Console.WriteLine("== Retrieving Memories Directly From the Store ==");
var vdbResult2 = await memoryStore.GetWithPointIdAsync(memoryCollectionName, myOldFB);
Console.WriteLine(vdbResult2 != null ? vdbResult2.Metadata.Text : "ERROR: memory not found");

== Retrieving Memories Through the Kernel ==
I currently work in the hotel industry at the front desk. I won the best team player award.
== Retrieving Memories Directly From the Store ==
Can you believe the size of the trees in Yellowstone? They're huge! I'm so committed to forestry concerns.


---
### **Retrieve Memories from Collection by filter (aka where clause)**

In [14]:
Console.WriteLine("== Similarity Searching Memories: My favorite work is ==");
var searchResults = kernel.Memory.SearchAsync(memoryCollectionName, "My work information", limit: 3, minRelevanceScore: 0.8, withEmbeddings: true);
//var searchResults = kernel.Memory.SearchAsync(memoryCollectionName, "My work information", limit: 3, minRelevanceScore: 0.8);

var relatedMemory = "I know nothing.";
var counter = 0;

await foreach (var item in searchResults)
{
    if (counter == 0) { relatedMemory = item.Metadata.Text; }
    Console.WriteLine(item.Metadata.Text + " : " + item.Relevance);           
    item.Embedding.Display(); 
}

== Similarity Searching Memories: My favorite work is ==
I have worked as a tourist operator for 8 years. I have also worked as a banking associate for 3 years. : 0.8027108


Vector,"[ -0.025853308, -0.032508742, 0.035322, -0.014170478, -0.025723064, 0.028419102, -0.020891035, 0.008296502, -0.015694326, -0.015772471, -0.0039887032, 0.0011111387, -0.006922435, -0.0036696068, 0.006066085, -0.0022271615, 0.014886816, -0.012692216, -0.011116271, -0.00092391385 ... (1516 more) ]"
IsEmpty,False
Count,1536


I currently work in the hotel industry at the front desk. I won the best team player award. : 0.8023426


Vector,"[ -0.021766275, -0.0010293088, 0.0073690643, -0.025581928, -0.027194731, 0.038051642, -0.017924396, 0.012089461, 0.0006359424, -0.019183168, 0.014908587, 0.006251248, 0.01042421, -0.012692623, -0.0055464664, -0.012994204, 0.0240478, -0.008332811, -0.010712679, -0.014856138 ... (1516 more) ]"
IsEmpty,False
Count,1536


---
### **Use Vector DB results in a Semantic Function (aka Skill or Plugin)**

In [15]:
var myFunction = kernel.CreateSemanticFunction(@"
{{$input}}
Tell me about me and my work history in less than 70 characters.
", maxTokens: 100, temperature: 0.1, topP: .1);

var result = await myFunction.InvokeAsync(relatedMemory);

Console.WriteLine("Response from AI: " +result);

Response from AI: I'm a hotel front desk worker, winner of the best team player award.


---
### **Create a Collection and then delete it**

In [16]:
//Add test collection and delete it
const string memoryCollectionDelete = "Memory to Delete";

var delMoon = await kernel.Memory.SaveInformationAsync(memoryCollectionDelete, id: "Moon distance",
    text: "The distance from the Earth to the moon is approximately 238,855 miles (384,400 kilometers)");
var delParis = await kernel.Memory.SaveInformationAsync(memoryCollectionDelete, id: "Paris NYC distance",
    text: "The distance between Paris and New York City is approximately 3,631 miles");


Console.WriteLine("== Printing Collections in DB ==");
var theCollections = memoryStore.GetCollectionsAsync();
await foreach (var collection in theCollections)
{
    Console.WriteLine(collection);
}

Console.WriteLine("== Removing Collection {0} ==", memoryCollectionDelete);
await memoryStore.DeleteCollectionAsync(memoryCollectionDelete);

Console.WriteLine("== Printing Collections in DB ==");
await foreach (var collection in theCollections)
{
    Console.WriteLine(collection);
}

== Printing Collections in DB ==
Memory to Delete
Facts About Me
Facts About Me2
== Removing Collection Memory to Delete ==
== Printing Collections in DB ==
Facts About Me
Facts About Me2
